In [1]:
import sys
sys.path.append('../')
from utils import *
from model import *

In [2]:
data_dir = '/datadrive/data_cs/'
batch_size = 128
batch_num  = 128
epoch_num  = 1000
samp_num   = 64 - 1

device = torch.device("cuda:2")
graph = dill.load(open(data_dir + 'graph.pk', 'rb'))

In [3]:
train_range = {t: True for t in graph.times if t != None and t < 2015}
valid_range = {t: True for t in graph.times if t != None and t >= 2015  and t <= 2016}
test_range  = {t: True for t in graph.times if t != None and t > 2016}

In [4]:
class GNN(nn.Module):
    def __init__(self, in_dim, n_hid, num_types, num_relations, n_heads, n_layers, dropout = 0.3):
        super(GNN, self).__init__()
        self.gcs = nn.ModuleList()
        self.num_types = num_types
        self.in_dim    = in_dim
        self.n_hid     = n_hid
        self.aggregat_ws   = nn.ModuleList()
        self.drop          = nn.Dropout(dropout)
        for t in range(num_types):
            self.aggregat_ws.append(nn.Linear(in_dim, n_hid))
        for l in range(n_layers):
            self.gcs.append(RAGCNConv(n_hid, n_hid, num_types, num_relations, n_heads, dropout))
    def set_device(self, device):
        self.device = device
        for gc in self.gcs:
            gc.device = device
    def forward(self, node_feature, node_type, edge_time, edge_index, edge_type):
        res = torch.zeros(node_feature.size(0), self.n_hid).to(node_feature.device)
        for t_id in range(self.num_types):
            aggregat_w = self.aggregat_ws[t_id]
            idx = (node_type == t_id)
            if idx.sum() == 0:
                continue
            res[idx] = torch.tanh(aggregat_w(node_feature[idx]))
        meta_xs = self.drop(res)
        del res
        for gc in self.gcs:
            meta_xs = gc(meta_xs, node_type, edge_index, edge_type, edge_time)
        return meta_xs

In [5]:
def multi_mask(task_set, feature, time, edge_list, batch_size):
    rem_lists   = []
    neg_nums    = []
    ori_lists   = []
    for target_type, source_type, rel_type in task_set:
        edges = np.array(edge_list[target_type][source_type][rel_type])
        '''
            edge: (target_id, source_id)
        '''
        ori_lists += [edges]
        remn  = min((len(edges)-1) // 2, batch_size)
        rem_ids = np.random.choice(np.arange(len(edges)), remn, replace = False)
        lft_ids = np.array([i for i in range(len(edges)) if i not in rem_ids])
        lft_edges = edges[lft_ids]
        rem_lists += [edges[rem_ids]]
        neg_nums  += [len(time[target_type])]
        edge_list[target_type][source_type][rel_type] = list(lft_edges)
        edge_list[source_type][target_type]['rev_' + rel_type] = list(np.stack((lft_edges[:,1], lft_edges[:,0])).T)
    node_feature, node_type, node_time, edge_index, edge_type, node_dict, _ = to_torch(feature, time, edge_list, graph)
    return neg_nums, rem_lists, node_feature, node_type, node_time, edge_index, edge_type, node_dict, ori_lists
def mt_sample(seed, time_range, task_set, sampled_depth = 5, sampled_number = 100, batch_size = batch_size):
    np.random.seed(seed)
    train_feature, train_time, train_edge_list, _, _ = \
            sample_subgraph(graph, time_range=time_range, sampled_depth = sampled_depth, sampled_number = sampled_number)
    return multi_mask(task_set, train_feature, train_time, train_edge_list, batch_size)

def prepare_data(pool, process_ids, task_set):
    jobs = []
    for process_id in process_ids[:-1]:
        p = pool.apply_async(mt_sample, args=(np.random.randint(2**32 - 1), train_range, task_set))
        jobs.append(p)
    p = pool.apply_async(mt_sample, args=(np.random.randint(2**32 - 1), valid_range, task_set))
    jobs.append(p)
    return jobs
def neg_sample(size, pos_id):
    s = np.arange(size)
    np.random.shuffle(s)
    return [si for si in s if si not in pos_id]

In [6]:
class Matcher(nn.Module):
    '''
        Matching between a pair of nodes to conduct link prediction.
        Use multi-head attention as matching model.
    '''
    def __init__(self, n_hid, n_heads, dropout = 0.5):
        super(Matcher, self).__init__()
        self.left_linear    = nn.Linear(n_hid,  n_hid)
        self.right_linear   = nn.Linear(n_hid,  n_hid)
        self.merge          = nn.Linear(n_heads, 1)
        self.drop     = nn.Dropout(dropout)
        self.n_heads  = n_heads
        self.dk       = n_hid // n_heads
        self.cache      = None
    def forward(self, x, y, infer = False, pair = False):
        ty = self.drop(self.right_linear(y)).view(-1, self.n_heads, self.dk)
        if infer:
            '''
                During testing, we will consider millions or even billions of nodes as candidates (x).
                It's not possible to calculate them again for different query (y)
                Since the model is fixed, we propose to cache them, and dirrectly use the results.
            '''
            if self.cache != None:
                tx = self.cache
            else:
                tx = self.left_linear(x)
                self.cache = tx.view(-1, self.n_heads, self.dk)
        else:
            tx = (self.left_linear(x)).view(-1, self.n_heads, self.dk)
        if pair:
            res = self.drop(tx * ty).sum(dim=-1)
        else:
            res = torch.matmul(tx.transpose(0, 1), ty.transpose(0, 1).transpose(-1, -2)).transpose(0, 2)
        return self.merge(res).squeeze()

In [7]:
types = graph.get_types()
gnn = GNN(in_dim = len(graph.node_feature['paper']['emb'][0]) + 401, n_hid = 256, num_types = len(types), \
          num_relations = len(graph.get_meta_graph()) + 1, n_heads = 8, n_layers = 4).to(device)
fp_predictor = Matcher(256, 8).to(device)
vp_predictor = Matcher(256, 8).to(device)
pp_predictor = Matcher(256, 8).to(device)
ap_predictor = Matcher(256, 8).to(device)
ai_predictor = Matcher(256, 8).to(device)
model = nn.Sequential(gnn, fp_predictor, vp_predictor, pp_predictor, ap_predictor, ai_predictor)

In [8]:
optimizer = torch.optim.AdamW(model.parameters())
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, 1000, eta_min=1e-6)

In [ ]:
stats = []
models = [fp_predictor, vp_predictor, pp_predictor, ap_predictor, ai_predictor]
task_set = [['field', 'paper', 'PF_in_L2'],\
            ['venue', 'paper', 'PV_Conference'],\
            ['paper', 'paper', 'PP_cite'],\
            ['paper', 'author', 'AP_write_first'],\
            ['affiliation', 'author', 'in'],\
           ]

pool = mp.Pool(8)
process_ids = np.arange(batch_num // 8)
st = time.time()
jobs = prepare_data(pool, process_ids, task_set)
train_step = 1500
best_val   = 0

for epoch in np.arange(epoch_num)+1:
    train_data = [job.get() for job in jobs[:-1]]
    valid_data = jobs[-1].get()
    pool.close()
    pool.join()
    pool = mp.Pool(8)
    jobs = prepare_data(pool, process_ids, task_set)
    et = time.time()
    print('Data Preparation: %.1fs' % (et - st))
    
    train_losses = []
    model.train()
    torch.cuda.empty_cache()
    for neg_nums, rem_lists, node_feature, node_type, node_time, edge_index, edge_type, node_dict, ori_lists in train_data:
        node_emb = gnn.forward(node_feature.to(device), node_type.to(device), \
                        node_time.to(device), edge_index.to(device), edge_type.to(device))
        losses = []
        for target_size, predictor, rem_edges, ori_edges, (target_type, source_type, rel_type) in \
                zip(neg_nums, models, rem_lists, ori_lists, task_set):
            '''
                Train
            '''
            positive_target_ids, source_ids = rem_edges[:,0].reshape(-1, 1), rem_edges[:,1]
            negative_target_ids = [neg_sample(target_size, \
                ori_edges[:, 0][ori_edges[:, 1] == s_id].tolist()) for  s_id in source_ids]
            sn = np.min([len(_id) for _id in negative_target_ids] + [samp_num])
            negative_target_ids = np.array([negative_target_id[:sn] for negative_target_id in negative_target_ids])
            source_ids = source_ids + node_dict[source_type][0]
            target_ids = np.concatenate((positive_target_ids, negative_target_ids), axis=-1) + node_dict[target_type][0]
            source_emb = node_emb[source_ids].repeat(1, target_ids.shape[1]\
                                              ).view(target_ids.shape[0] * target_ids.shape[1], -1)
            target_emb = node_emb[target_ids.reshape(-1)].view(target_ids.shape[0] * target_ids.shape[1], -1)
            res = predictor.forward(source_emb, target_emb, pair=True).view(target_ids.shape[0], target_ids.shape[1])
            losses += [-torch.log_softmax(res, dim=-1)[:,0]]
        loss = torch.cat(losses).mean()
        optimizer.zero_grad() 
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.2)
        optimizer.step()
        torch.cuda.empty_cache()
        train_losses += [loss.cpu().detach().tolist()]
        train_step += 1
        scheduler.step(train_step)
    '''
        Valid
    '''
    model.eval()
    with torch.no_grad():
        neg_nums, rem_lists, node_feature, node_type, node_time, edge_index, edge_type, node_dict, ori_lists = valid_data
        valid_losses = []
        valid_accs   = []
        node_emb = gnn.forward(node_feature.to(device), node_type.to(device), \
                        node_time.to(device), edge_index.to(device), edge_type.to(device))
        for target_size, predictor, rem_edges, ori_edges, (target_type, source_type, rel_type) in \
                zip(neg_nums, models, rem_lists, ori_lists, task_set):
            '''
                Valid
            '''
            positive_target_ids, source_ids = rem_edges[:,0].reshape(-1, 1), rem_edges[:,1]
            negative_target_ids = [neg_sample(target_size, \
                ori_edges[:, 0][ori_edges[:, 1] == s_id].tolist()) for  s_id in source_ids]
            sn = np.min([len(_id) for _id in negative_target_ids] + [samp_num])
            negative_target_ids = np.array([negative_target_id[:sn] for negative_target_id in negative_target_ids])
            source_ids = source_ids + node_dict[source_type][0]
            target_ids = np.concatenate((positive_target_ids, negative_target_ids), axis=-1) + node_dict[target_type][0]
            source_emb = node_emb[source_ids].repeat(1, target_ids.shape[1]\
                                              ).view(target_ids.shape[0] * target_ids.shape[1], -1)
            target_emb = node_emb[target_ids.reshape(-1)].view(target_ids.shape[0] * target_ids.shape[1], -1)
            res = predictor.forward(source_emb, target_emb, pair=True).view(target_ids.shape[0], target_ids.shape[1])
            valid_losses += [-torch.log_softmax(res, dim=-1)[:,0]]
            s = (res.argmax(dim=1) == 0).sum()
            valid_accs += [s.tolist() / batch_size]
        valid_losses = torch.cat(valid_losses).cpu().detach().tolist()
        s = (res.argmax(dim=1) == 0).sum()
        st = time.time()
        print(("Epoch: %d (%.1fs)  LR: %.5f Train Loss: %f  Valid Loss: %f  Valid Acc: %f") % \
              (epoch, (st-et), optimizer.param_groups[0]['lr'], np.average(train_losses), np.average(valid_losses), np.average(valid_accs)))
        if np.average(valid_accs) > best_val:
            best_val = np.average(valid_accs)
            torch.save(gnn, './save/mt_model.pt')
        stats += [[train_losses, valid_losses]]

Data Preparation: 24.1s
Epoch: 1 (38.0s)  LR: 0.00052 Train Loss: 8.558474  Valid Loss: 4.768923  Valid Acc: 0.014063


/home/ziniu/anaconda3/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type GNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/ziniu/anaconda3/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type ModuleList. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/ziniu/anaconda3/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type RAGCNConv. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/ziniu/anaconda3/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Linear. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won

Data Preparation: 4.1s
Epoch: 2 (42.1s)  LR: 0.00055 Train Loss: 7.517610  Valid Loss: 4.847802  Valid Acc: 0.017188


/home/ziniu/anaconda3/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type GNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/ziniu/anaconda3/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type ModuleList. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/ziniu/anaconda3/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type RAGCNConv. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/ziniu/anaconda3/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Linear. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won

Data Preparation: 4.8s
Epoch: 3 (46.5s)  LR: 0.00057 Train Loss: 6.943894  Valid Loss: 4.844967  Valid Acc: 0.021875


...

/home/ziniu/anaconda3/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type GNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/ziniu/anaconda3/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type ModuleList. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/ziniu/anaconda3/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type RAGCNConv. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/ziniu/anaconda3/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Linear. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won

Data Preparation: 5.3s
Epoch: 443 (48.4s)  LR: 0.00095 Train Loss: 3.008860  Valid Loss: 3.115239  Valid Acc: 0.201563
Data Preparation: 3.7s
Epoch: 444 (46.0s)  LR: 0.00094 Train Loss: 2.955381  Valid Loss: 3.119895  Valid Acc: 0.142187
Data Preparation: 5.5s
Epoch: 445 (48.1s)  LR: 0.00093 Train Loss: 3.022627  Valid Loss: 3.102524  Valid Acc: 0.189062
Data Preparation: 5.1s
Epoch: 446 (67.2s)  LR: 0.00091 Train Loss: 2.994807  Valid Loss: 3.111229  Valid Acc: 0.171875
Data Preparation: 5.4s
Epoch: 447 (50.1s)  LR: 0.00090 Train Loss: 2.970821  Valid Loss: 3.077706  Valid Acc: 0.181250
Data Preparation: 4.3s
Epoch: 448 (46.1s)  LR: 0.00089 Train Loss: 2.965047  Valid Loss: 3.151102  Valid Acc: 0.164062
Data Preparation: 4.9s
Epoch: 449 (49.6s)  LR: 0.00087 Train Loss: 2.981311  Valid Loss: 2.997872  Valid Acc: 0.215625
Data Preparation: 5.0s
Epoch: 450 (52.8s)  LR: 0.00085 Train Loss: 2.985012  Valid Loss: 3.165371  Valid Acc: 0.168750
Data Preparation: 4.9s
Epoch: 451 (50.1s)  LR: 0

Data Preparation: 5.1s
Epoch: 512 (49.5s)  LR: 0.00008 Train Loss: 2.880503  Valid Loss: 3.012297  Valid Acc: 0.204687
Data Preparation: 4.3s
Epoch: 513 (48.9s)  LR: 0.00009 Train Loss: 2.856710  Valid Loss: 2.968652  Valid Acc: 0.204687
Data Preparation: 4.5s
Epoch: 514 (52.0s)  LR: 0.00011 Train Loss: 2.867367  Valid Loss: 3.172644  Valid Acc: 0.154688
Data Preparation: 4.2s
Epoch: 515 (49.7s)  LR: 0.00012 Train Loss: 2.874218  Valid Loss: 3.031451  Valid Acc: 0.189062
Data Preparation: 4.3s
Epoch: 516 (50.0s)  LR: 0.00014 Train Loss: 2.929089  Valid Loss: 2.943818  Valid Acc: 0.212500
Data Preparation: 4.9s
Epoch: 517 (48.0s)  LR: 0.00015 Train Loss: 2.880819  Valid Loss: 3.225380  Valid Acc: 0.162500
Data Preparation: 4.6s
Epoch: 518 (49.9s)  LR: 0.00017 Train Loss: 2.895757  Valid Loss: 3.030242  Valid Acc: 0.206250
Data Preparation: 4.8s
Epoch: 519 (48.9s)  LR: 0.00019 Train Loss: 2.867600  Valid Loss: 3.049527  Valid Acc: 0.178125
Data Preparation: 5.0s
Epoch: 520 (48.7s)  LR: 0

/home/ziniu/anaconda3/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type GNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/ziniu/anaconda3/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type ModuleList. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/ziniu/anaconda3/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type RAGCNConv. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/ziniu/anaconda3/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Linear. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won

Data Preparation: 4.8s
Epoch: 535 (53.1s)  LR: 0.00054 Train Loss: 2.832420  Valid Loss: 3.227523  Valid Acc: 0.157812
Data Preparation: 4.9s
Epoch: 536 (49.1s)  LR: 0.00056 Train Loss: 2.886307  Valid Loss: 3.044426  Valid Acc: 0.179688
Data Preparation: 4.3s
Epoch: 537 (48.7s)  LR: 0.00059 Train Loss: 2.858059  Valid Loss: 3.147645  Valid Acc: 0.164062
Data Preparation: 4.3s
Epoch: 538 (54.5s)  LR: 0.00061 Train Loss: 2.895253  Valid Loss: 3.055039  Valid Acc: 0.214062
Data Preparation: 4.6s
Epoch: 539 (50.9s)  LR: 0.00063 Train Loss: 2.909330  Valid Loss: 2.892218  Valid Acc: 0.221875
Data Preparation: 4.7s
Epoch: 540 (50.3s)  LR: 0.00065 Train Loss: 2.900515  Valid Loss: 3.035559  Valid Acc: 0.209375
Data Preparation: 5.1s
Epoch: 541 (47.4s)  LR: 0.00068 Train Loss: 2.856420  Valid Loss: 3.135521  Valid Acc: 0.164062
Data Preparation: 5.2s
Epoch: 542 (48.8s)  LR: 0.00070 Train Loss: 2.878989  Valid Loss: 2.998194  Valid Acc: 0.214062
Data Preparation: 4.7s
Epoch: 543 (48.8s)  LR: 0

In [ ]:
train_loss = []
test_loss  = []
for s in stats:
    train_loss += [np.average(s[0])]
    test_loss += [np.average(s[1])]

In [ ]:
plt.plot(train_loss, label='train')
plt.plot(test_loss, label='test')
plt.legend()
plt.show()

In [ ]:
for target_size, predictor, rem_edges, ori_edges, (target_type, source_type, rel_type) in \
        zip(neg_nums, models, rem_lists, ori_lists, task_set):
    '''
        Valid
    '''
    positive_target_ids, source_ids = rem_edges[:,0].reshape(-1, 1), rem_edges[:,1]
    negative_target_ids = [neg_sample(target_size, \
        ori_edges[:, 0][ori_edges[:, 1] == s_id].tolist()) for  s_id in source_ids]
    sn = np.min([len(_id) for _id in negative_target_ids] + [samp_num])
    negative_target_ids = np.array([negative_target_id[:sn] for negative_target_id in negative_target_ids])
#     source_ids = source_ids + node_dict[source_type][0]
#     target_ids = np.concatenate((positive_target_ids, negative_target_ids), axis=-1) + node_dict[target_type][0]

In [ ]:
negative_target_ids.shape

In [ ]:
np.concatenate((positive_target_ids, negative_target_ids), axis=-1).shape

In [ ]:
node_emb

In [ ]:
node_emb[target_ids.reshape(-1)]

In [ ]:
gnn = torch.load('./save/mt_model.pt')

In [ ]:
size = 0
for s in graph.edge_list:
    for t in graph.edge_list[s]:
        for e in graph.edge_list[s][t]:
            for i in graph.edge_list[s][t][e]:
                size += len(graph.edge_list[s][t][e][i])
size
size = 0
for s in graph.node_feature:
    size += len(graph.node_feature[s])
size

In [ ]:
size = 0
for s in graph.node_feature:
    size += len(graph.node_feature[s])
size

In [ ]:
epoch

In [ ]:
np.unique(list(graph.node_feature['venue']['attr']))

In [ ]:
graph.node_feature['venue'][graph.node_feature['venue']['attr']=='Patent']

In [ ]:
1116163, 5574903, 

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import euclidean_distances

In [ ]:
topconf = graph.node_feature['venue'].nlargest(256, 'citation')

In [ ]:
res = list(graph.node_feature['venue'][graph.node_feature['venue']['name'] == 'WWW']['emb'])

In [ ]:
cos = euclidean_distances(res, list(topconf['emb']))[0]
for li in np.argsort(cos)[:50]:
    print(list(topconf['name'])[li])

In [ ]:
gnn.to(device)
gnn.eval()
topconf = graph.node_feature['venue'].nlargest(100, 'citation')
with torch.no_grad():
    _time = 2000
    vids = np.stack([graph.node_feature['venue'].nlargest(100, 'citation').index.values, np.repeat([_time], 100)]).T
    conf_emb1 = []
    conf_emb2 = []
    for i in range(10):
        print(i)
        feature, times, edge_list, _ = sample_subgraph(graph, {t: True for t in graph.times if t != None}, \
                        inp = {'venue': vids}, sampled_depth = 4, sampled_number = 256)
        node_feature, node_type, edge_time, edge_index, edge_type, node_dict, edge_dict = \
                    to_torch(feature, times, edge_list, graph)
        venue_ids = np.arange(len(vids)) + node_dict['venue'][0]

        node_feature, node_type, edge_time, edge_index, edge_type = node_feature.to(device), node_type.to(device), \
                              edge_time.to(device), edge_index.to(device), edge_type.to(device)
        res = torch.zeros(node_feature.size(0), gnn.n_hid).to(node_feature.device)
        for t_id in range(gnn.num_types):
            aggregat_w = gnn.aggregat_ws[t_id]
            idx = (node_type == t_id)
            if idx.sum() == 0:
                continue
            res[idx] = torch.tanh(aggregat_w(node_feature[idx]))
        meta_xs = gnn.drop(res)
        del res
        meta_xs = gnn.gcs[0](meta_xs, node_type, edge_index, edge_type, edge_time)
        emb = meta_xs[venue_ids].cpu().detach().numpy()
        conf_emb1 += [emb]
        cos = cosine_similarity(emb)
        for li in np.argsort(-cos[list(topconf['name']).index('KDD')])[:50]:
            print(list(topconf['name'])[li])
        print('-' * 100)
        dis = euclidean_distances(emb)
        for li in np.argsort(dis[list(topconf['name']).index('KDD')])[:50]:
            print(list(topconf['name'])[li])
        print('=' * 100)
        
        
        meta_xs = gnn.gcs[1](meta_xs, node_type, edge_index, edge_type, edge_time)
        emb = meta_xs[venue_ids].cpu().detach().numpy()
        conf_emb2 += [emb]
        cos = cosine_similarity(emb)
        for li in np.argsort(-cos[list(topconf['name']).index('KDD')])[:50]:
            print(list(topconf['name'])[li])
        print('-' * 100)
        dis = euclidean_distances(emb)
        for li in np.argsort(dis[list(topconf['name']).index('KDD')])[:50]:
            print(list(topconf['name'])[li])
        print('=' * 100)

In [ ]:
coss = [euclidean_distances(emb) for emb in conf_emb1]
cos = np.average(coss, axis=0)
for li in np.argsort(cos[list(topconf['name']).index('ACL')])[:50]:
    print(list(topconf['name'])[li])
print('-' * 100)

In [ ]:
coss = [euclidean_distances(emb) for emb in conf_emb2]
cos = np.average(coss, axis=0)
for li in np.argsort(cos[list(topconf['name']).index('ACL')])[:50]:
    print(list(topconf['name'])[li])
print('-' * 100)

In [ ]:
time_emb = {2010: [conf_emb1, conf_emb2]}

In [ ]:
time_emb[1990] = [conf_emb1, conf_emb2]

In [ ]:
time_emb[2030] = [conf_emb1, conf_emb2]

In [ ]:
time_emb[1990][1]

In [ ]:
node_dict = {j: i for i, j in enumerate(graph.get_types())}

In [ ]:
edge_dict = {e[2]: i for i, e in enumerate(graph.get_meta_graph())}
edge_dict['self'] = len(edge_dict)

In [ ]:
_s = 0
for i in graph.edge_list['author']['affiliation']['rev_in']:
    _s += len(graph.edge_list['author']['affiliation']['rev_in'][i])

In [ ]:
_s

In [ ]:
len(graph.edge_list['author']['affiliation']['rev_in'])

In [ ]:
gnn, _ = torch.load('../paper-field-L2/save/rgt_1.pt')

In [ ]:

for i, j, k in graph.get_meta_graph():
    if i == 'affiliation':
        print(i,j,k)
        print(gnn.gcs[0].relation_ws[node_dict[i]][edge_dict[k]][node_dict[j]].mean())

In [ ]:
paper venue rev_PV_Conference
tensor(1.0509, device='cuda:1', grad_fn=<MeanBackward0>)

